In [25]:
# import torch

# # Simulated predictions and targets (replace with actual data)
# predictions = torch.tensor([0, 1, 2, 0, 2, 1])
# targets = torch.tensor([0, 1, 2, 1, 2, 0])

# # Calculate the multiclass confusion matrix
# confusion_matrix = torch.zeros(3, 3)

# for t, p in zip(targets, predictions):
#     confusion_matrix[t, p] += 1

# # Print the confusion matrix
# print("Confusion Matrix:")
# print(confusion_matrix)

In [3]:
import os
import sys
from PIL import Image

from tensorboardX import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

import seaborn as sn
import pandas as pd

import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.io import read_image
from torch.utils.data import Dataset

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from super_gradients.training import Trainer, models
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val
)
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import (
    PPYoloEPostPredictionCallback
)

sys.stdout = sys.__stdout__

In [4]:
class config:
    #trainer params
    HOME = os.getcwd()

    CHECKPOINT_DIR = f'{HOME}\checkpoint\AGI-Dataset' #specify the path you want to save checkpoints to
    EXPERIMENT_NAME = 'AGIExperiment' 

    ##dataset params
    DATA_DIR = f'{HOME}\GearInspection-Dataset3\CategoryNG\ClassAll' 
    LOGS = f'{CHECKPOINT_DIR}\AGILogs'

    # CATEGORY = 'CategoryNG\ClassAll'

    TRAIN_IMAGES_DIR = 'train\images' 
    TRAIN_LABELS_DIR = 'train\labels' 

    VAL_IMAGES_DIR = 'valid\images'
    VAL_LABELS_DIR = 'valid\labels' 

    TEST_IMAGES_DIR = 'test\images' 
    TEST_LABELS_DIR = 'test\labels'

    #what class names do you have
    CLASSES = ['akkon', 'dakon', 'kizu', 'hakkon', 'kuromoyou', 'mizunokori', 'senkizu', 'yogore']

    NUM_CLASSES = len(CLASSES)

    DATALOADER_PARAMS={
    'batch_size':64,
    'num_workers':2
    }

    EPOCHS = 1
    RUNNING_LOSS = 0.0
    ACCURACY = 0.0

    # model params
    MODEL_NAME = 'yolo_nas_l' # choose from yolo_nas_s, yolo_nas_m, yolo_nas_l
    PRETRAINED_WEIGHTS = 'coco' 

In [3]:
# Define a custom dataset class to load images and labels from separate folders
class CustomImageDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_filenames = os.listdir(images_dir)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.images_dir, self.image_filenames[idx])
        label_name = os.path.join(self.labels_dir, self.image_filenames[idx] + '.txt')  # Assuming labels are in text files

        print(label_name)

        image = read_image(img_name)
        
        # Load and process the label file (adjust as needed based on your label file format)
        with open(label_name, 'r') as label_file:
            label = label_file.read().strip()
            label = int(label)  # Assuming labels are integer values

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
# Define your data transformation
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# Create custom datasets for train, validation, and test
train_dataset = CustomImageDataset(
    images_dir=f'{config.DATA_DIR}\\{config.TRAIN_IMAGES_DIR}',
    labels_dir=f'{config.DATA_DIR}\\{config.TRAIN_LABELS_DIR}',
    transform=transform
)

val_dataset = CustomImageDataset(
    images_dir=f'{config.DATA_DIR}\\{config.VAL_IMAGES_DIR}',
    labels_dir=f'{config.DATA_DIR}\\{config.VAL_LABELS_DIR}',
    transform=transform
)

test_dataset = CustomImageDataset(
    images_dir=f'{config.DATA_DIR}\\{config.TEST_IMAGES_DIR}',
    labels_dir=f'{config.DATA_DIR}\\{config.TEST_LABELS_DIR}',
    transform=transform
)

In [5]:
train_root = f'{config.DATA_DIR}\\{config.TRAIN_IMAGES_DIR}'

print("train_root")

In [8]:
# Create custom datasets for train, validation, and test
train_dataset = ImageFolder(
    root=train_root,
    transform=transform
)

val_dataset = ImageFolder(
    root=config.VAL_IMAGES_DIR,
    transform=transform
)

test_dataset = ImageFolder(
    root=config.TEST_IMAGES_DIR,
    transform=transform
)

FileNotFoundError: Couldn't find any class folder in c:\Users\mhajj\Documents\RUTILEA\mhajjaj\GearInspection-Dataset3\CategoryNG\ClassAll\train\images.